In [1]:
from IPython.display import display, HTML

display(HTML(data="""

<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

%reload_ext jupyternotify

#%notify

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import sys

import qutip as qtp
import matplotlib.pyplot as plt
from pathlib import Path

from scipy import linalg as la

parent_path = ".."  #str(Path(__file__).resolve().parents[1])
sys.path.append(parent_path)

import pysqkit


import time
import os

from pysqkit.solvers.solvkit import integrate

%matplotlib notebook
plt.rcParams['figure.figsize'] = (10, 7)

In [3]:
from math import pi

h = 6.62607 * 10**(-34)
h_bar = h/(2*pi)
e = 1.6071766*10**(-19)
phi_0 = h/(2*e)



In [4]:
N = 100

# Init

In [5]:
J_c = 300*10**6 *h
lam = .5

In [6]:
t = np.linspace(0, 1000, 20000) #* 10**(-9) #in nano seconds

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

## flxA



$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

In [7]:
e_c_A = 1*10**9 *h
e_l_A = 1.5*10**9 *h
e_j_A = 3.8*10**9 *h
  
flux_A = 1/2  # 1/2 -> pi dans le cos

flxA = pysqkit.qubits.Fluxonium(label = "QBA",
                               charge_energy=e_c_A,
                               induct_energy=e_l_A,
                               joseph_energy=e_j_A,
                               flux=flux_A,
                               basis=None,
                               dim_hilbert=N)

In [8]:
flxA.diagonlize_basis(num_levels=5)

In [9]:
flxA.hamiltonian(as_qobj = True)

Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[ 2.07104840e-24 -3.27116402e-40 -6.76614217e-40 -2.63797023e-41
   2.50341245e-39]
 [-3.27116402e-40  2.83416879e-24  1.10098039e-39  1.44514164e-38
  -7.72046669e-40]
 [-1.76297870e-39  1.10098039e-39  5.00783767e-24 -2.94141630e-40
  -8.74411250e-39]
 [-2.63797023e-41  1.47780289e-38 -2.94141630e-40  7.16338839e-24
  -7.93414787e-40]
 [ 2.58128059e-39 -7.72046669e-40 -7.79952613e-39 -7.93414787e-40
   9.60985683e-24]]

In [10]:
#Bare energies

bare_E_A = flxA.eig_energies(levels = 5)
bare_E_A/(h*10**9)

array([ 3.12560598,  4.2772998 ,  7.55777961, 10.81091566, 14.50310188])

In [11]:
# transitions freqs

omega_01_A = bare_E_A[1] - bare_E_A[0]
omega_12_A = bare_E_A[2] - bare_E_A[1]

omega_01_A/(h*10**9) , omega_12_A/(h*10**9) 

(1.1516938223021471, 3.2804798038301657)

In [12]:
# mat elements :

n_10_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_A = np.abs(flxA.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxA.mat_elements('charge_op', levels = 5)).data, 2), n_01_A, n_10_A

(array([[0.  , 0.25, 0.  , 0.26, 0.  ],
        [0.25, 0.  , 0.61, 0.  , 0.18],
        [0.  , 0.61, 0.  , 0.75, 0.  ],
        [0.26, 0.  , 0.75, 0.  , 0.94],
        [0.  , 0.18, 0.  , 0.94, 0.  ]]),
 0.24866384335580397,
 0.24866384335580402)

### Rq)

In [ ]:
plt.figure()
for i in range(len(bare_E_A)):
    plt.scatter(i, bare_E_A[i])

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.charge_zpf*(create(100) - destroy(100))*1j

In [ ]:
flxA.charge_op(as_qobj = True)

In [ ]:
flxA.hamiltonian(as_qobj = True)/(h*10**9)

## flxB

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

In [13]:
e_c_B = 1*10**9 *h
e_l_B = .9*10**9 *h
e_j_B = 3*10**9 *h

flux_B = 1/2

flxB = pysqkit.qubits.Fluxonium(label = "QBB",
                               charge_energy=e_c_B,
                               induct_energy=e_l_B,
                               joseph_energy=e_j_B,
                               flux=flux_B,
                               basis=None,
                               dim_hilbert=N)

In [14]:
flxB.diagonlize_basis(num_levels=5)

In [15]:
#Bare energies

bare_E_B = flxB.eig_energies(levels = 5)
bare_E_B/(h*10**9)

array([ 2.07847477,  2.92730143,  5.85594017,  8.53943723, 11.62680359])

In [16]:
# transitions freqs

omega_01_B = bare_E_B[1] - bare_E_B[0]
omega_12_B = bare_E_B[2] - bare_E_B[1]

omega_01_B/(h*10**9) , omega_12_B/(h*10**9) 

(0.848826660531331, 2.9286387379969727)

In [17]:
# mat elements :

n_10_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[1,0]
n_01_B = np.abs(flxB.mat_elements('charge_op', levels = 5)).data[0,1]

np.round(np.abs(flxB.mat_elements('charge_op', levels = 5)).data, 2), n_01_B, n_10_B

(array([[0.  , 0.21, 0.  , 0.28, 0.  ],
        [0.21, 0.  , 0.57, 0.  , 0.17],
        [0.  , 0.57, 0.  , 0.67, 0.  ],
        [0.28, 0.  , 0.67, 0.  , 0.86],
        [0.  , 0.17, 0.  , 0.86, 0.  ]]),
 0.20722487336294823,
 0.20722487336294826)

## coupling


$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

In [18]:
g = J_c#J_c/(4*e**2)

In [19]:
duo = flxA.couple_to(flxB, pysqkit.couplers.capacitive_coupling, strength = g)

In [20]:
duo.eig_energies()/(h*10**9)

array([ 5.20384121,  6.05143224,  6.35584504,  7.20120932,  8.9812114 ,
        9.63511118, 10.1012304 , 10.5096234 , 11.66444338, 12.78707325,
       12.88814298, 13.39053209, 13.77923186, 14.753792  , 15.80925828,
       16.15034663, 16.58062423, 16.67231987, 17.46371008, 19.0507949 ,
       19.45204948, 20.39850888, 22.35870751, 23.13521456, 26.13904644])

In [21]:
H_duo_bare = duo.bare_hamiltonian(as_qobj = True)
H_duo_int = duo.int_hamiltonian(as_qobj = True)


In [22]:
H_duo_tot = qtp.Qobj(H_duo_bare.full() + H_duo_int.full(), dims = H_duo_bare.dims,  #dims = [[25], [25]]
                 shape=H_duo_bare.shape,
                type='oper',
                isherm=True)


In [23]:
H_duo_tot

Quantum object: dims = [[5, 5], [5, 5]], shape = (25, 25), type = oper, isherm = True
Qobj data =
[[ 3.44826033e-24 -5.12053397e-40 -3.13728510e-38 -3.27505439e-40
  -4.62539668e-38 -3.27116402e-40  1.02431091e-26 -2.46617410e-41
  -1.37077372e-26  1.64020814e-41 -6.76614217e-40  1.15370178e-41
  -2.77770100e-56 -1.54392973e-41  1.84739909e-56 -2.63797023e-41
  -1.07174725e-26  2.58038383e-41  1.43425492e-26 -1.71616698e-41
   2.50341245e-39 -7.31857251e-42  1.76205035e-56  9.79400562e-42
  -1.17190806e-56]
 [-5.12053397e-40  4.01069882e-24 -1.01535364e-39 -1.04954656e-38
  -8.36507238e-40 -1.02431091e-26 -3.27116402e-40  2.80022822e-26
   1.78034220e-41 -8.55355550e-27 -1.15370178e-41 -6.76614217e-40
   3.15395281e-41  2.00523487e-56 -9.63403989e-42  1.07174725e-26
  -2.63797023e-41 -2.92990816e-26 -1.86279072e-41  8.94967484e-27
   7.31857251e-42  2.50341245e-39 -2.00072781e-41 -1.27203209e-56
   6.11140772e-42]
 [-3.13033187e-38 -1.01535364e-39  5.95123535e-24 -3.89982140e-40
  -1.9

### rqs

In [ ]:
duo.bare_hamiltonian(), duo.int_hamiltonian()

In [ ]:
#Bare energies

bare_E_duo = la.eig(duo.bare_hamiltonian())[0].real/h #in Hz
bare_E_duo.sort()
bare_E_duo = bare_E_duo[:5]*10**(-9)  #in GHz

int_E_duo = la.eig(duo.int_hamiltonian())[0].real/h
int_E_duo.sort()
int_E_duo = int_E_duo[:5]*10**(-9)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (10,5))

axs[0].scatter(range(len(bare_E_duo)), bare_E_duo, c = 'r', label = 'bare')    
axs[1].scatter(range(len(int_E_duo)), int_E_duo, c = 'b', label = 'int')

axs[0].legend()
axs[1].legend()

axs[0].set_ylabel("E (GHz)")
axs[1].set_ylabel("E (GHz)")

In [ ]:
E_duo = bare_E_duo + int_E_duo
E_duo.sort()

plt.figure()
plt.scatter(range(len(E_duo)), E_duo)

plt.annotate("  |00>", (0,E_duo[0]))
plt.annotate("  |01>", (1,E_duo[1]))
plt.annotate("  |10>", (2,E_duo[2]))
plt.annotate("  |11>", (3,E_duo[3]))
plt.annotate("  |??>", (4,E_duo[4]))




plt.ylabel('E (GHz)')

In [ ]:
print("Ground =", E_duo[0], "GHz\n1st state =", E_duo[1], "GHz\n2nd state =", E_duo[2], "GHz" )

print("\n\nomega 00->01 =", E_duo[1]-E_duo[0], "GHz\nomega 01->10 =", E_duo[2] - E_duo[1], "GHz\nomega 10->11 =", E_duo[3]-E_duo[2], "GHz"
     )

In [ ]:
duo.hamiltonian()

## Drive

$$\hat{H}_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \cdot \left( \eta_A \cdot \hat{n}_A + \eta_B \cdot \hat{n}_B \right)$$

$$f\left( t \right) = f_0 \cdot \dfrac{f\left( t \right)}{f_0}$$

In [24]:
f_0 = lam/2 * np.abs(omega_01_A - omega_01_B) / (n_10_A)

In [25]:
t_rise = 25*10**(-9)
sigma = t_rise/2

f_by_f0 = np.exp(- (t-t_rise)**2 / (2*sigma**2)) - np.exp(-t_rise**2 /(2*sigma**2))*np.ones(len(t))

In [26]:
f = f_0 * f_by_f0

omega_d = 2*pi * 1.15 *10**9
gamma_d = 0

eta_1 = 1
eta_2 = 1

In [27]:
n_A = flxA.charge_op(as_qobj = True)
n_B = flxB.charge_op(as_qobj = True)

In [30]:
drive = qtp.tensor(n_A, qtp.qeye(5)) + qtp.tensor(qtp.qeye(5), n_B)

In [31]:
drive

Quantum object: dims = [[5, 5], [5, 5]], shape = (25, 25), type = oper, isherm = True
Qobj data =
[[0.+0.j         0.-0.20722487j 0.+0.j         0.+0.27731659j
  0.+0.j         0.+0.24866384j 0.+0.j         0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.-0.26017959j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j        ]
 [0.+0.20722487j 0.+0.j         0.-0.56650469j 0.+0.j
  0.+0.17304409j 0.+0.j         0.+0.24866384j 0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.-0.26017959j 0.+0.j         0.+0.j         0.+0.j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j        ]
 [0.+0.j         0.+0.56650469j 0.+0.j         0.+0.67240126j
  0.+0.j         0.+0.j         0.+0.j         0.+0.24866384j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.+0.j         0.+0.j 

In [32]:
pulse = 2 * f * np.cos(omega_d*t)

### Rq)

# Simu

##  With Solver

In [34]:
time =  t
state_init = qtp.tensor(qtp.Qobj(np.ones(5)/np.sqrt(5)), qtp.Qobj(np.array([1,0,0,0,0])))  
#the first is in superposition of 0 through 4 and the second at ground

hamil = H_duo_tot
drive_solver = [drive]
pulse_solver = [pulse]
jump = []
solver = pysqkit.solvers.solvkit.supported_solvers[0]

In [35]:
state_init

Quantum object: dims = [[5, 5], [1, 1]], shape = (25, 1), type = ket
Qobj data =
[[0.4472136]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.4472136]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.4472136]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.4472136]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.4472136]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]]

In [36]:
res = integrate(time, state_init, hamil, drive_solver, pulse_solver, jump, solver)

In [37]:
res.states[-1]

Quantum object: dims = [[5, 5], [1, 1]], shape = (25, 1), type = ket
Qobj data =
[[ 4.47213595e-01]
 [-4.23102543e-24]
 [-1.81281047e-47]
 [ 5.66212697e-24]
 [-7.21388035e-47]
 [ 4.47213595e-01]
 [-4.23102543e-24]
 [-1.81281047e-47]
 [ 5.66212697e-24]
 [-7.21388035e-47]
 [ 4.47213595e-01]
 [-4.23102543e-24]
 [-1.81281047e-47]
 [ 5.66212697e-24]
 [-7.21388035e-47]
 [ 4.47213595e-01]
 [-4.23102543e-24]
 [-1.81281047e-47]
 [ 5.66212697e-24]
 [-7.21388035e-47]
 [ 4.47213595e-01]
 [-4.23102543e-24]
 [-1.81281047e-47]
 [ 5.66212697e-24]
 [-7.21388035e-47]]

# Interpretation

In [51]:
mat = qtp.fock(25,0) * qtp.fock(25,0).trans()

op = qtp.Qobj(mat)

expect_val = np.zeros(len(t)) 

for k in range(len(t)):
    expect_val[k] = qtp.expect(op, res.states[k])

TypeError: Invalid operand types

In [50]:
mat

Quantum object: dims = [[25], [25]], shape = (25, 25), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 